In [ ]:
import tensorflow as tf
import numpy as np
import scipy
import h5py
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import set_random_seed
from tensorflow.python.keras.preprocessing  import  image 
from tensorflow.python.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense,Conv2D, MaxPooling2D,ELU
from tensorflow.python.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from IPython.display import display
from numpy.random import seed
tf.set_random_seed(1)
seed(1)

In [ ]:
# dimensions of our images.
WORK_DIRECTORY="C:/tmp/dogcat_study/"
os.chdir(WORK_DIRECTORY) 
train_data_dir      = 'KaggleCatDog/train'
validation_data_dir = 'KaggleCatDog/validation'
weightsFilePath= "weights/weightscatsdogs.h5"
modelFilePath= "model/modelcatsdogs.json"

path = WORK_DIRECTORY+"UserData/"
#Hyperparameters
nb_train_samples = 24158
nb_validation_samples = 1052
epochs =3
batch_size = 16
num_classes =2
image_width, image_height = 32, 32
input_shape = (image_width, image_height, 3)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./ 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1,   activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
filepath=weightsFilePath+"{epoch:03d}-{val_acc:3f}.h5"
checkpointer = ModelCheckpoint(filepath=weightsFilePath,
                               verbose=1, 
                               monitor='val_acc',
                               save_best_only=True,
                               mode='max')

In [ ]:
with tf.device('/gpu:0'):
    history=model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples,
        callbacks=[checkpointer])        

model_json = model.to_json()
with open(modelFilePath, "w") as json_file:
    json_file.write(model_json)     

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss','val_loss'])
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(acc)
plt.plot(val_acc)
plt.legend(['acc','val_acc'])
plt.title("Accuracy")
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
files = os.listdir(path)

for file in files:
    pred=-1
    t_image= image.load_img(path + file )
    print(path + file )
    test_image = image.load_img(path + file, target_size=(image_height, image_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    pred = model.predict_on_batch(test_image)
  
    if pred >= 1.0:
        print("Dog")
    else:
        print("Cat")
    plt.imshow(t_image)
    plt.show()      